In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV

In [2]:
players = pd.read_csv("players.csv")
squads = pd.read_csv("squads.csv")

In [3]:
df = players.join(squads.set_index(["squad_name", "year"]), how="left", on=["squad_name", "year"], lsuffix="_player", rsuffix="_squad")

num = df._get_numeric_data()
num[num < 0] = pd.NA

In [4]:
df.describe()

,age,value_player,year,games_played,goals,assists,minute_played,goals_conceded,clean_sheets,avg_age,squad_value,number_players
count,42118.000000,4.211800e+04,42118.000000,42118.000000,37417.000000,37417.000000,42118.000000,4701.000000,4701.000000,42118.000000,4.211800e+04,42118.000000
mean,24.390142,5.631461e+06,2015.430695,24.164015,2.973675,2.125157,1730.675151,22.058073,4.928951,25.343775,2.052651e+08,37.577710
std,4.834412,1.090845e+07,3.373213,12.595201,4.612372,2.923134,1063.986822,19.439744,5.220464,1.105396,2.065252e+08,7.883551
min,-1.000000,-1.000000e+00,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,1.840000e+07,21.000000
25%,20.000000,5.000000e+05,2013.000000,15.000000,0.000000,0.000000,887.000000,5.000000,1.000000,24.600000,7.150000e+07,33.000000
50%,24.000000,2.000000e+06,2015.000000,25.000000,1.000000,1.000000,1682.000000,18.000000,3.000000,25.300000,1.230300e+08,36.000000
75%,28.000000,5.500000e+06,2018.000000,34.000000,4.000000,3.000000,2508.000000,36.000000,8.000000,26.000000,2.543300e+08,41.000000
max,43.000000,2.000000e+08,2021.000000,75.000000,73.000000,35.000000,6498.000000,91.000000,32.000000,30.100000,1.200000e+09,88.000000


In [5]:
any(df['championship_squad'] != df['championship_player'])

False

Since there are no values where player plays in different championship than squad, we drop the column:

In [6]:
df = players.join(squads.drop('championship', axis=1).set_index(["squad_name", "year"]), how="left", on=["squad_name", "year"])

Define the preprocessing transformations

For numeric:
- Impute values using the median (this may not be the best aproach)
- Scale the data with a StandardScaler

For categorical variables we perform a OneHotEncoding (ignoring unknowns in case there is a malformed call to the model)

Finally, we use CountVectorizer on text variables (player name)

In [7]:
numeric_transformer = Pipeline(
    steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
          ]
)

# If a category has low frequency on training, label it as infrequent, when running
# the model a category is not found it will be asigned as infrequent or ignored.
categorical_transformer = OneHotEncoder(
    handle_unknown="infrequent_if_exist",
    min_frequency=0.001,
)

categorical_columns = ["squad_name", "role", "championship"]

column_trans = make_column_transformer(
    (CountVectorizer(), "name_player"),
    (categorical_transformer, categorical_columns),
    remainder=numeric_transformer
)

In [8]:
df.role = df.role.astype("object")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42118 entries, 0 to 42117
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name_player     42118 non-null  object 
 1   age             42118 non-null  int64  
 2   role            42118 non-null  object 
 3   value_player    42118 non-null  int64  
 4   squad_name      42118 non-null  object 
 5   year            42118 non-null  int64  
 6   games_played    42118 non-null  int64  
 7   goals           37417 non-null  float64
 8   assists         37417 non-null  float64
 9   minute_played   42118 non-null  int64  
 10  championship    42118 non-null  object 
 11  goals_conceded  4701 non-null   float64
 12  clean_sheets    4701 non-null   float64
 13  avg_age         42118 non-null  float64
 14  squad_value     42118 non-null  int64  
 15  number_players  42118 non-null  int64  
dtypes: float64(5), int64(7), object(4)
memory usage: 5.1+ MB


Simple XGBoost on R2 with a train test split of 8:2

In [9]:
clf = Pipeline(
    steps=[("preprocessor", column_trans), ("classifier", XGBRegressor(objective='reg:squarederror'))]
)
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder=Pipeline(steps=[('imputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('scaler',
                                                              StandardScaler())]),
                                   transformers=[('countvectorizer',
                                                  CountVectorizer(),
                                                  'name_player'),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=0.001),
                                                  ['squad_name', 'role',
                                                   'championship'])])),
                ('c...
                              gamma=None, gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=None,
                              max_leaves=None, min_child_weight=None,
                              missing=nan, monotone_constraints=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, predictor=None,
                              random_state=None, reg_alpha=None,
                              reg_lambda=None, ...))])

Preliminary results

In [10]:
X = df.drop("value_player", axis=1)
y = df.value_player

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.735


In [11]:
len(clf.steps[0][1].get_feature_names_out())

12013

In [12]:
scores = cross_val_score(clf, X, y)
(-scores)

array([-0.65556625, -0.6962115 , -0.76181871, -0.74748747, -0.66949606])

## Parameter tunning

We have the following parameters to adjust:

In [13]:
clf.named_steps["classifier"].get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

In [14]:
params = {
    #"booster" : ("gbtree", "gblinear"),
    "eta" : [0.01, 0.1],
    "gamma": [0, 0.1, 0.2],
    "max_depth": [5,6,7,8],
}

# Build new pipeline with cross validation (reusing all the preprocessing, since we won't be touching that
gcv = Pipeline(
    steps=[("preprocessor", clf.named_steps["preprocessor"]),
           ("classifier", GridSearchCV(clf.named_steps["classifier"], params))]
)

In [15]:
gcv.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder=Pipeline(steps=[('imputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('scaler',
                                                              StandardScaler())]),
                                   transformers=[('countvectorizer',
                                                  CountVectorizer(),
                                                  'name_player'),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=0.001),
                                                  ['squad_name', 'role',
                                                   'championship'])])),
                ('c...
                                                     interaction_constraints='',
                                                     learning_rate=0.300000012,
                                                     max_bin=256,
                                                     max_cat_to_onehot=4,
                                                     max_delta_step=0,
                                                     max_depth=6, max_leaves=0,
                                                     min_child_weight=1,
                                                     missing=nan,
                                                     monotone_constraints='()',
                                                     n_estimators=100, n_jobs=0,
                                                     num_parallel_tree=1,
                                                     predictor='auto',
                                                     random_state=0,
                                                     reg_alpha=0, reg_lambda=1, ...),
                              param_grid={'eta': [0.01, 0.1],
                                          'gamma': [0, 0.1, 0.2],
                                          'max_depth': [5, 6, 7, 8]}))])

In [16]:
gcv.named_steps["classifier"].best_params_, gcv.named_steps["classifier"].best_score_

({'eta': 0.01, 'gamma': 0, 'max_depth': 7}, 0.7442533147721287)

In [17]:
gcv.named_steps["classifier"].cv_results_

{'mean_fit_time': array([4.71344328, 4.45599561, 5.23437905, 7.00824494, 3.64202986,
        4.6046905 , 5.89721642, 5.234971  , 3.37889042, 3.54351912,
        3.43980284, 3.86026816, 3.20606956, 3.83710337, 6.97791309,
        5.94163828, 3.53684964, 5.28222346, 5.29329681, 7.71366096,
        4.20265994, 4.66856952, 5.16825953, 6.43141289]),
 'std_fit_time': array([1.71907683, 0.66331488, 1.94891238, 3.66884875, 0.87582799,
        2.05971788, 1.96154287, 0.85070992, 0.1256279 , 0.74498654,
        0.09861167, 0.05399831, 0.85381461, 0.85589599, 3.71864093,
        1.31207937, 0.46847237, 2.89107275, 2.1314303 , 3.64609498,
        0.55690922, 0.34946606, 0.64235136, 0.5676801 ]),
 'mean_score_time': array([0.01112328, 0.01797471, 0.01304002, 0.01545706, 0.00889211,
        0.01716814, 0.02010226, 0.0139617 , 0.01580725, 0.01066298,
        0.01161904, 0.01621323, 0.00891542, 0.01004415, 0.01264582,
        0.01181884, 0.0133173 , 0.02061753, 0.01188121, 0.01892614,
        0.013137

In [18]:
player_data = dict(X_test.iloc[0,:])

gcv.predict(
pd.DataFrame.from_dict({k: [v] for k, v in player_data.items()})
)[0]

5289734.5

In [19]:
dict(X_test.iloc[0,:])

{'name_player': 'Aritz Aduriz',
 'age': 32,
 'role': 'Centre-Forward',
 'squad_name': 'Athletic Bilbao',
 'year': 2013,
 'games_played': 36,
 'goals': 18.0,
 'assists': 9.0,
 'minute_played': 2693,
 'championship': 'primera-division',
 'goals_conceded': nan,
 'clean_sheets': nan,
 'avg_age': 25.8,
 'squad_value': 130699999,
 'number_players': 27}

In [20]:
y_test.iloc[0]

3500000

In [21]:
gcv.score(X_test, y_test)

0.7342334958380897

In [22]:
gcv

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder=Pipeline(steps=[('imputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('scaler',
                                                              StandardScaler())]),
                                   transformers=[('countvectorizer',
                                                  CountVectorizer(),
                                                  'name_player'),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=0.001),
                                                  ['squad_name', 'role',
                                                   'championship'])])),
                ('c...
                                                     interaction_constraints='',
                                                     learning_rate=0.300000012,
                                                     max_bin=256,
                                                     max_cat_to_onehot=4,
                                                     max_delta_step=0,
                                                     max_depth=6, max_leaves=0,
                                                     min_child_weight=1,
                                                     missing=nan,
                                                     monotone_constraints='()',
                                                     n_estimators=100, n_jobs=0,
                                                     num_parallel_tree=1,
                                                     predictor='auto',
                                                     random_state=0,
                                                     reg_alpha=0, reg_lambda=1, ...),
                              param_grid={'eta': [0.01, 0.1],
                                          'gamma': [0, 0.1, 0.2],
                                          'max_depth': [5, 6, 7, 8]}))])